# 04. Layer Normalization 구현

## 학습 목표
- Layer Normalization을 **직접 구현**
- Batch Norm vs Layer Norm 차이 이해
- 왜 Transformer에서 Layer Norm을 쓰는지

## 관련 면접 질문
- Batch Normalization과 Layer Normalization의 차이는?
- Transformer에서 Layer Norm을 쓰는 이유는?
- Pre-LN vs Post-LN의 차이는?

---
## Pre-Quiz

### Q1. Batch Norm은 어느 축으로 정규화하나요? Layer Norm은?
```
Batch Norm: 
Layer Norm: 
```

### Q2. NLP에서 Batch Norm 대신 Layer Norm을 쓰는 이유는?
```
당신의 답: 
```

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
torch.manual_seed(42)

---
## 1. Layer Normalization 구현

### 수식
$$y = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} \cdot \gamma + \beta$$

- $\mu$: 마지막 차원의 평균
- $\sigma^2$: 마지막 차원의 분산
- $\gamma$, $\beta$: 학습 가능한 파라미터

In [ ]:
# =================================================
# TODO: Layer Normalization 구현
# =================================================
#
# $$\hat{x} = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}}$$
# $$y = \gamma \hat{x} + \beta$$
#
# - μ, σ²: 마지막 축에서 계산 (각 sample별 독립)
# - γ, β: learnable parameters (shape = normalized_shape)
#
# Batch Norm과 차이: 정규화 축이 다름!
#
# =================================================

class LayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-6):
        """
        Args:
            normalized_shape: 정규화할 차원 크기 (보통 d_model)
            eps: 분모 안정화
        """
        super().__init__()
        
        if isinstance(normalized_shape, int):
            normalized_shape = (normalized_shape,)
        
        # ============================================
        # γ, β 파라미터 정의
        # ============================================
        
        pass
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """(..., normalized_shape) → 동일 shape"""
        # ============================================
        # 위 수식 구현 (mean, var는 dim=-1)
        # ============================================
        
        pass

In [ ]:
# ============ 테스트 ============
def test_layer_norm():
    batch, seq, d_model = 2, 10, 64
    
    # 직접 구현
    our_ln = LayerNorm(d_model)
    # PyTorch 구현
    pytorch_ln = nn.LayerNorm(d_model)
    
    # 동일한 가중치 설정
    pytorch_ln.weight.data = our_ln.gamma.data.clone()
    pytorch_ln.bias.data = our_ln.beta.data.clone()
    
    x = torch.randn(batch, seq, d_model)
    
    our_output = our_ln(x)
    pytorch_output = pytorch_ln(x)
    
    assert our_output.shape == x.shape, f"Shape 오류: {our_output.shape}"
    assert torch.allclose(our_output, pytorch_output, atol=1e-5), "PyTorch와 불일치!"
    
    # 정규화 확인: 마지막 축의 mean ≈ 0, std ≈ 1
    mean = our_output.mean(dim=-1)
    std = our_output.std(dim=-1)
    
    print("✅ Layer Norm 테스트 통과!")
    print(f"   Output mean (should ≈ 0): {mean.mean():.6f}")
    print(f"   Output std (should ≈ 1): {std.mean():.6f}")

test_layer_norm()

---
## 2. Batch Norm vs Layer Norm 시각화

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 데이터 (batch=4, features=3)
data = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
]

# Batch Norm: 세로 방향 (같은 feature across batch)
axes[0].imshow([[1], [1], [1], [1]], cmap='Blues', alpha=0.5, aspect='auto', extent=[-0.5, 0.5, 3.5, -0.5])
axes[0].imshow([[1], [1], [1], [1]], cmap='Greens', alpha=0.5, aspect='auto', extent=[0.5, 1.5, 3.5, -0.5])
axes[0].imshow([[1], [1], [1], [1]], cmap='Reds', alpha=0.5, aspect='auto', extent=[1.5, 2.5, 3.5, -0.5])

for i in range(4):
    for j in range(3):
        axes[0].text(j, i, str(data[i][j]), ha='center', va='center', fontsize=14)

axes[0].set_xticks([0, 1, 2])
axes[0].set_yticks([0, 1, 2, 3])
axes[0].set_xticklabels(['f1', 'f2', 'f3'])
axes[0].set_yticklabels(['batch1', 'batch2', 'batch3', 'batch4'])
axes[0].set_title('Batch Normalization\n각 feature를 batch 방향으로 정규화')

# Layer Norm: 가로 방향 (같은 sample의 모든 features)
axes[1].imshow([[1, 1, 1]], cmap='Blues', alpha=0.5, aspect='auto', extent=[-0.5, 2.5, 0.5, -0.5])
axes[1].imshow([[1, 1, 1]], cmap='Greens', alpha=0.5, aspect='auto', extent=[-0.5, 2.5, 1.5, 0.5])
axes[1].imshow([[1, 1, 1]], cmap='Oranges', alpha=0.5, aspect='auto', extent=[-0.5, 2.5, 2.5, 1.5])
axes[1].imshow([[1, 1, 1]], cmap='Reds', alpha=0.5, aspect='auto', extent=[-0.5, 2.5, 3.5, 2.5])

for i in range(4):
    for j in range(3):
        axes[1].text(j, i, str(data[i][j]), ha='center', va='center', fontsize=14)

axes[1].set_xticks([0, 1, 2])
axes[1].set_yticks([0, 1, 2, 3])
axes[1].set_xticklabels(['f1', 'f2', 'f3'])
axes[1].set_yticklabels(['batch1', 'batch2', 'batch3', 'batch4'])
axes[1].set_title('Layer Normalization\n각 sample을 feature 방향으로 정규화')

plt.tight_layout()
plt.show()

---
## Post-Quiz

### Q1. 가변 길이 시퀀스에서 Batch Norm이 문제인 이유는?
```
당신의 답: 
```

### Q2. Pre-LN (Attention 전에 LN) vs Post-LN의 차이는?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 보기</summary>

### Pre-Quiz

**Q1. 정규화 축**
> - Batch Norm: **batch 축** (같은 feature across samples)
> - Layer Norm: **feature 축** (같은 sample의 모든 features)

**Q2. NLP에서 Layer Norm 사용 이유**
> - **가변 길이**: 시퀀스 길이가 다르면 batch 통계가 불안정
> - **batch size 독립**: Layer Norm은 batch size 1에서도 동작
> - **inference 일관성**: 학습/추론 시 동일한 동작

---

### Post-Quiz

**Q1. 가변 길이에서 Batch Norm 문제**
> - padding된 위치도 통계에 포함 → 왜곡
> - 짧은 문장과 긴 문장의 통계가 섞임
> - running statistics가 불안정

**Q2. Pre-LN vs Post-LN**
> - **Post-LN (원래)**: LN(x + Attention(x)) - gradient 흐름 어려움
> - **Pre-LN**: x + Attention(LN(x)) - 학습 안정적, warm-up 불필요
> - 최근 모델들은 Pre-LN 선호

</details>

---
## 구현 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class LayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-6):
        super().__init__()
        
        if isinstance(normalized_shape, int):
            normalized_shape = (normalized_shape,)
        
        self.normalized_shape = normalized_shape
        self.eps = eps
        
        self.gamma = nn.Parameter(torch.ones(normalized_shape))
        self.beta = nn.Parameter(torch.zeros(normalized_shape))
    
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        
        x_norm = (x - mean) / torch.sqrt(var + self.eps)
        
        return self.gamma * x_norm + self.beta
```

</details>